In [1]:
import pandas as pd
import numpy as np
from docx import Document
from pathlib import Path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
from typing import List, Dict
from datetime import datetime
import sys
import os
import re

In [2]:

def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [3]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.word_averias_extractor import extract_averias_table

from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_df_word_datos_averias
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_datos_averias_corte_excel import merge_word_datos_averias_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.o1_averias_word_validator import validate_averias_word
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.o1_averias_word_validator import build_failure_messages_validate_averias_word


In [4]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 2.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_30-03-2025_06-04-2025_20250410_173936.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_30-03-2025_04-04-2025_20250410_195338.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS - CORTE 2.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS - CORTE 2.docx"



In [5]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335) 
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)
df_word_datos_averias =  extract_averias_table(DIR_WORD_DATOS)
df_word_telefonia_averias = extract_averias_table(DIR_WORD_TELEFONIA)



In [6]:
#df_word_datos_averias = df_word_datos_averias.rename(columns={'Tiempo\nTotal (HH:MM)': 'Tiempo Total (HH:MM)'})
df_process_word = preprocess_df_word_datos_averias(df_word_datos_averias)
summary = get_dataframe_summary(df_process_word)
summary

,Data Type,Non Null Count,Null Count,Null Percentage,Unique Values
N°,object,36,0,0.00,20
nro_incidencia,object,36,0,0.00,36
CUISMP,object,36,0,0.00,33
Fecha y Hora Inicio,object,36,0,0.00,36
Fecha y Hora Fin,object,36,0,0.00,26
Avería reportada,object,36,0,0.00,3
Causa,object,36,0,0.00,14
Tiempo real de afectación (HH:MM),object,36,0,0.00,36
Tiempo Total (HH:MM),object,36,0,0.00,36
Distrito Fiscal,object,36,0,0.00,18


In [7]:

df_preprocss_excel = preprocess_corte_excel(df_corte_excel)



In [8]:
merge_corte_word_averia_dato = merge_word_datos_averias_corte_excel(
    df_process_word,
    df_preprocss_excel,
      'both' )
#merge_corte_word_averia_dato.head(1)

In [9]:
#df_process_word

In [10]:
summa_word = get_dataframe_summary(df_process_word)
#summa_word

In [11]:
sum_excel = get_dataframe_summary(df_preprocss_excel)
sum_excel

,Data Type,Non Null Count,Null Count,Null Percentage,Unique Values
nro_incidencia,object,154,0,0.00,154
FECHA Y HORA INICIO,datetime64[ns],154,0,0.00,150
FECHA Y HORA FIN,datetime64[ns],154,0,0.00,125
ESTATUS,object,154,0,0.00,2
SERVICIO,object,154,0,0.00,4
CUISMP,object,154,0,0.00,110
TIPO CASO,object,154,0,0.00,5
AVERÍA,object,154,0,0.00,24
TIEMPO (HH:MM),object,154,0,0.00,140
COMPONENTE,object,154,0,0.00,3


In [12]:
row = df_preprocss_excel[df_preprocss_excel['nro_incidencia'] == '21798497']
#row

In [13]:
df_vali = validate_averias_word(merge_corte_word_averia_dato, 'COMPONENTE II')
df_vali

,N°,nro_incidencia,CUISMP_word_datos_averias,Fecha y Hora Inicio,Fecha y Hora Fin,Avería reportada,Causa,Tiempo real de afectación (HH:MM),Tiempo Total (HH:MM),Distrito Fiscal,responsable,FECHA Y HORA INICIO,FECHA Y HORA FIN,ESTATUS,SERVICIO,CUISMP_corte_excel,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,DIRECCION,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,USUARIO,ESPECIALISTA,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,DC + INDISPONIBILIDAD,TIEMPO (HH:MM)_trimed,FIN-INICIO (HH:MM)_trimed,_merge,Componente,cuismp_word_averia,FECHA_Y_HORA_INICIO_fmt,FECHA_Y_HORA_FIN_fmt,Fecha_hora_inicio_match,fecha_hora_fin_match,CUISMP_match,tipo_caso_match,averia_match,tiempo_hhmm_match,componente_match,df_match,fin_inicio_hhmm_match,responsabilidad_match,Validation_OK,fail_count
0,1,21789943,14008,18/03/2025 16:46,19/03/2025 12:54,SIN SERVICIO,BLOQUEO DE MODEM SATELITAL,06:32,20:08,Junin,CLARO,2025-03-18 16:46:00,2025-03-19 12:54:00,Caso Concluído,Red Privada Virtual Full Mesh,14008,SIN SERVICIO,BLOQUEO DE MODEM SATELITAL,11:49:00,COMPONENTE II,Junin,"SN . . SIN NOMBRE Jr. Huancayo Plaza Principal de Santo Domingo de Acobamba (FISCALIA , DML) / -11.768895,-74.795526",Se generó ticket para la revisión del servicio de datos de la sede Acobamba2,21092931,20:08:00,El inconveniente se originó debido a un problema con el equipo de red satelital ubicado en la sedel cliente.,CLARO,RECLAMO,11,Entre 8h a 24h,E756566,CLARO - CORTE,21789943,No,"El cliente, el Sr. Denis Carhuamaca, reportó problemas con el servicio de datos identificado con el CUISMP 14008 y se generó el ticket 21789943 el día 18/03/2025 a las 16:46 horas. Inmediatamente, Claro realizó las revisiones encontrando un inconveniente de conectividad con los equipos ubicados en la sede del cliente. Ante ello, intentamos comunicarnos con el cliente, el Sr. Denis Carhuamaca a las 17:24 horas, sin éxito. Posteriormente, a las 19:04 horas en comunicación el cliente, el Sr. Denis Carhuamaca, se realizó los descartes correspondientes de reinicio de equipos y validación de conexiones, sin superar el evento de pérdida de conectividad. Seguidamente, se gestionó el desplazamiento de personal técnico especializado a la sede del cliente para las revisiones correspondientes, durante la gestión de desplazamiento se verificó el restablecimiento del servicio el día 19/03/2025 a las 12:54 horas por restablecimiento en el equipo de red satelital ubicado en la sedel cliente. Finalmente, el desplazamiento de personal a la sede del cliente se revisará con el ticket 21790828.",REPORTE PREVIO - Calidad,"70,56",Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s) \n18/03/2025 22:41:00 hasta el día 19/03/2025 07:00:00\n(Total de horas sin acceso a la sede: 08:19 horas),El inconveniente se originó debido a un problema con el equipo de red satelital ubicado en la sedel cliente.,11:49,20:08,both,COMPONENTE II,14008,18/03/2025 16:46,19/03/2025 12:54,True,True,True,True,True,False,True,True,True,True,False,1
1,4,21793097,17004,24/03/2025 21:41,25/03/2025 08:00,SIN SERVICIO-NO DA TONO,TELEFONO / ANEXO AVERIADO,10:19,10:19,Lima,CLARO,2025-03-24 21:41:00,2025-03-25 08:00:00,Caso Concluído,Telefonía Fija,17004,SIN SERVICIO-NO DA TONO,TELEFONO / ANEXO AVERIADO,10:19,COMPONENTE IV,Lima,"JR AZANGARO 374 -12.048449, -77.028841",Se generó ticket para la revisión del servicio de datos de la sede Azangaro,20326886,10:19,El inconveniente se originó por un problema en la configuración de los anexos ubicados en la sede del cliente.,CLARO,RECLAMO,10,Entre 8h a 24h,E756566,CLARO - CORTE GESTIONADO,21793097,No,"El cliente la Srta. Zenaida Mollenedo, reportó inconvenientes con un anexo de la sede identificada con el CUISMP 17004, se generó ticket 21793097 el día 24/03/2025 a las 21:41 horas. Ante ello, se gestionó el desplazamiento de personal 

In [14]:
df_mess = build_failure_messages_validate_averias_word(df_vali)
df_mess

,nro_incidencia,mensaje,TIPO REPORTE,objetivo
0,21789943,No coincide TIEMPO (HH:MM) de WORD : 06:32 es diferente a Tiempo real de afectación (HH:MM) de Excel: 11:49.,RECLAMO,2.10
1,21793097,No coincide Componente de WORD : COMPONENTE II es diferente a COMPONENTE de Excel: COMPONENTE IV.,RECLAMO,2.10
2,21796282,No coincide CUISMP_word_telefonia de WORD : 0 es diferente a CUISMP_corte_excel: 6001. No coincide Distrito Fiscal de WORD : nan es diferente a DF de Excel: Cajamarca.,RECLAMO,2.10
3,21798497,No coincide TIEMPO (HH:MM) de WORD : 09:14 es diferente a Tiempo real de afectación (HH:MM) de Excel: 03:14.,RECLAMO,2.10
4,21791018,No coincide Causa de WORD : PROBLEMA LAN DEL CLIENTE es diferente a AVERÍA de Excel: SATURAMIENTO ANCHO DE BANDA ALQUILADO. No coincide TIEMPO (HH:MM) de WORD : 07:20 es diferente a Tiempo real de afectación (HH:MM) de Excel: 07:19.,RECLAMO,2.10
8,21796929,No coincide Fecha y Hora Inicio de WORD : 01/04/2025 07:54 es diferente a EXCEL-CORTE: 01/04/2025 08:01. No coincide TIEMPO (HH:MM) de WORD : 00:04 es diferente a Tiempo real de afectación (HH:MM) de Excel: 00:07. No coincide Tiempo Total (HH:MM) de WORD : 00:04 es diferente a FIN-INICIO (HH:MM) de Excel: 00:11.,RECLAMO,2.10
10,21796956,No coincide TIEMPO (HH:MM) de WORD : 03:48 es diferente a Tiempo real de afectación (HH:MM) de Excel: 05:37.,RECLAMO,2.10
16,21790828,No coincide Fecha y Hora Inicio de WORD : 21/03/2025 11:07 es diferente a EXCEL-CORTE: 21/03/2025 23:17. No coincide TIEMPO (HH:MM) de WORD : 26:39 es diferente a Tiempo real de afectación (HH:MM) de Excel: 38:49. No coincide Tiempo Total (HH:MM) de WORD : 26:39 es diferente a FIN-INICIO (HH:MM) de Excel: 38:49.,RECLAMO,2.10
30,21798322,No coincide TIEMPO (HH:MM) de WORD : 03:47 es diferente a Tiempo real de afectación (HH:MM) de Excel: 03:46.,RECLAMO,2.10
